# GovTech TAP VICA Tech Assessment (Insurance Analysis)

In [21]:
from pprint import pprint

import pandas as pd
from sklearn import preprocessing

## EDA and Data Cleaning

### Load dataset

In [22]:
df_raw = pd.read_csv("./insurance_data.csv", sep=";")
df_raw.head(3)

,insuree#,gender,is45OrOlder,isMarried,hasKids,insuredMonths,termLifeInsurance,multipleTermLifePolicies,healthInsurance,healthRiders,premiumFrequency,eStatements,monthlyPremium,totalPremium,renewal
0,1,F,0.0,Yes,Yes,23,Yes,No,No,NaN,12.0,Yes,"19,65","451,55",Y
1,2,F,1.0,No,No,42,Yes,Yes,Class A,3,1.0,Yes,"84,65","3541,35",N
2,3,F,0.0,Yes,No,72,Yes,No,No,NaN,12.0,No,"19,4","1496,45",Y


**Thinking process** <br>

At a first look at the data we have, I found that "insuree#" column is just insuree id and is not relevant to our analysis or modelling. We can drop this column.

In [23]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5500 entries, 0 to 5499
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   insuree#                  5500 non-null   int64  
 1   gender                    5387 non-null   object 
 2   is45OrOlder               5404 non-null   float64
 3   isMarried                 5500 non-null   object 
 4   hasKids                   5500 non-null   object 
 5   insuredMonths             5500 non-null   int64  
 6   termLifeInsurance         5500 non-null   object 
 7   multipleTermLifePolicies  5500 non-null   object 
 8   healthInsurance           5500 non-null   object 
 9   healthRiders              3518 non-null   object 
 10  premiumFrequency          5386 non-null   float64
 11  eStatements               5500 non-null   object 
 12  monthlyPremium            5500 non-null   object 
 13  totalPremium              5500 non-null   object 
 14  renewal 

**Thinking process** <br>

- We can see that there are missing values for variables "gender", "is45OrOlder", "premiumFrequency" and "healthRiders"
- For variable "healthRiders", rows with missing value indicates that an insuree does not have any riders on his health insurance policy
- Given little domain knowledge about insurance industry and no extra context/information given, I decide to drop rows with missing values for variables "gender", "is45OrOlder" and "premiumFrequency" instead of filling with values (such as mean/median or 0 value) to prevent adding inaccurate information and noise to the data

### Drop irrelavant columns and rows with missing values

In [32]:
# Drop "insuree#" column
df = df_raw.drop(["insuree#"], axis=1)

# Drop rows with missing values, except for "heathRiders" variable
df = df.dropna(subset=["gender", "is45OrOlder", "premiumFrequency"])

print(f"Number of rows dropped: {len(df_raw) - len(df)}")

df.info()

Number of rows dropped: 320
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5180 entries, 0 to 5499
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gender                    5180 non-null   object 
 1   is45OrOlder               5180 non-null   float64
 2   isMarried                 5180 non-null   object 
 3   hasKids                   5180 non-null   object 
 4   insuredMonths             5180 non-null   int64  
 5   termLifeInsurance         5180 non-null   object 
 6   multipleTermLifePolicies  5180 non-null   object 
 7   healthInsurance           5180 non-null   object 
 8   healthRiders              3309 non-null   object 
 9   premiumFrequency          5180 non-null   float64
 10  eStatements               5180 non-null   object 
 11  monthlyPremium            5180 non-null   object 
 12  totalPremium              5180 non-null   object 
 13  renewal                   5180 non-

**Thinking process** <br>

- Number of rows dropped: 320
- Variables "monthlyPremium" and "totalPremium" should be of integer data type
- For categorical variables, we can print all the unique values for each variable to have a better understanding

### Get unique values for categorical variables

In [25]:
# Look at the values for "healthRiders" variable
print(df["healthRiders"].unique())

[nan '3' '3,4' '2' '1' '2,3,4' '1,2,3,4' '2,4' '1,2,4' '1,4' '1,2' '1,2,3'
 '1,3,4' '1,3' '4' '2,3']


**Thinking process** <br>

- After checking the values for "healthRiders", it seems like there 4 kinds of health rider, can perform one hot encoding

In [26]:
# Creating lists for continuous and categorical variables 
continuous_vars = ["insuredMonths", "monthlyPremium", "totalPremium"] 
cat_vars = list(set(list(df.columns)).difference(set(continuous_vars)))

print("Unique values for categorical variables")
print(f"'variable name': [values]\n")
for var in cat_vars:
    print(f"'{var}': {list(df[var].unique())}")

Unique values for categorical variables
'variable name': [values]

'healthInsurance': ['No', 'Class A', 'Class B']
'hasKids': ['Yes', 'No']
'healthRiders': [nan, '3', '3,4', '2', '1', '2,3,4', '1,2,3,4', '2,4', '1,2,4', '1,4', '1,2', '1,2,3', '1,3,4', '1,3', '4', '2,3']
'multipleTermLifePolicies': ['No', 'Yes', 'no term life policy taken', 'term life policy not taken']
'renewal': ['Y', 'N']
'premiumFrequency': [12.0, 1.0, 3.0]
'eStatements': ['Yes', 'No']
'isMarried': ['Yes', 'No']
'termLifeInsurance': ['Yes', 'No']
'gender': ['F', 'M']
'is45OrOlder': [0.0, 1.0]


**Thinking process** <br>

- Can perform label encoding for all the variables other than "heathRiders"
- Looking at variable "multipleTermLifePolicies", the values ['no term life policy taken', 'term life policy not taken'] means "No". To replace these values before label encoding

### Label encoding
- In order to keep track which label correspond to which value, we will create a value to label ID mapping

In [27]:
# Clean up the values in "multipleTermLifePolicies" variable
df["multipleTermLifePolicies"].replace('term life policy not taken', "No", inplace=True)
df["multipleTermLifePolicies"].replace('no term life policy taken', "No", inplace=True)

In [28]:
val2id = {}

# Remove "healthRiders" variable from category list
# since we will not perform label encoding on it
cat_vars.remove("healthRiders") 

for var in cat_vars:
    if var != "healthRiders":
        le = preprocessing.LabelEncoder()
        df[var] = le.fit_transform(df[var])
        val2id[var] = dict(zip(le.classes_, le.transform(le.classes_)))

pprint(val2id)

{'eStatements': {'No': 0, 'Yes': 1},
 'gender': {'F': 0, 'M': 1},
 'hasKids': {'No': 0, 'Yes': 1},
 'healthInsurance': {'Class A': 0, 'Class B': 1, 'No': 2},
 'is45OrOlder': {0.0: 0, 1.0: 1},
 'isMarried': {'No': 0, 'Yes': 1},
 'multipleTermLifePolicies': {'No': 0, 'Yes': 1},
 'premiumFrequency': {1.0: 0, 3.0: 1, 12.0: 2},
 'renewal': {'N': 0, 'Y': 1},
 'termLifeInsurance': {'No': 0, 'Yes': 1}}


### One hot encoding

In [29]:
# Create new columns for "healthRider" categories
for i in range(1, 5):
    df[f"healthRider{i}"] = 0
    df[f"healthRider{i}"] = df[f"healthRider{i}"].astype("category")
    cat_vars.append(f"healthRider{i}")  # Update category variable list

def health_rider_onehot_encode(row):
    """
    Helper function to perform one hot encoding on "healthRiders" variable
    """
    health_rider_str = row["healthRiders"]

    if type(health_rider_str) == str:
        health_riders = health_rider_str.split(",")
        for hr in health_riders:
            row[f"healthRider{hr}"] = 1
    
    return row
      
df = df.apply(lambda x: health_rider_onehot_encode(x), axis=1)
df = df.drop(["healthRiders"], axis=1)  # Remove "healthRiders" because not needed anymore

### Convert variables to continuous data type

In [30]:
df["monthlyPremium"] = df.apply(lambda x: int(x["monthlyPremium"].replace(',', '')) if x["monthlyPremium"] != " " else 0, axis=1)
df["totalPremium"] = df.apply(lambda x: int(x["totalPremium"].replace(',', '')) if x["totalPremium"] != " " else 0, axis=1)

In [31]:
df.head()

,gender,is45OrOlder,isMarried,hasKids,insuredMonths,termLifeInsurance,multipleTermLifePolicies,healthInsurance,premiumFrequency,eStatements,monthlyPremium,totalPremium,renewal,healthRider1,healthRider2,healthRider3,healthRider4
0,0,0,1,1,23,1,0,2,2,1,1965,45155,1,0,0,0,0
1,0,1,0,0,42,1,1,0,0,1,8465,354135,0,0,0,1,0
2,0,0,1,0,72,1,0,2,2,0,194,149645,1,0,0,0,0
3,0,0,1,1,13,1,0,2,2,0,1955,2653,1,0,0,0,0
4,0,0,0,0,37,1,1,0,0,0,1003,35414,1,0,0,1,1


**Thinking process** <br>

- Everything looks good now!
- Now we can answer guiding questions 1 and 2